In [ ]:
!pip install evaluate

In [ ]:

import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
from evaluate import load
import random
import re
import os
import warnings
from typing import List, Dict, Tuple

warnings.filterwarnings("ignore")

random.seed(42)
torch.manual_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
!pip install transformers
!pip install bitsandbytes
!pip install accelerate

In [ ]:
!pip install -U bitsandbytes

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import warnings

warnings.filterwarnings("ignore", category=UserWarning, message="The use of device_map=.* is deprecated")
warnings.filterwarnings("ignore", category=UserWarning, message="You are loading a model requiring `trust_remote_code=True`")

model_name = "microsoft/phi-2"
max_tokens = 500
Context_format = "{speaker}: {utterance}"
PHI_2_max_content = 2048


quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

try:
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=quantization_config,
        device_map="auto",
        trust_remote_code=True
    )

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    model.eval()

    print("Model and Tokenizer loaded successfully.")

except Exception as e:
    print("Ensure you have installed bitsandbytes and accelerate.")
    raise

def generate_response(prompt: str) -> str:

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=PHI_2_max_content).to(device)


    max_length = min(inputs['input_ids'].shape[1] + max_tokens, PHI_2_max_content)

    output_ids = model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        max_length=max_length,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        num_return_sequences=1,
        pad_token_id=tokenizer.pad_token_id
    )

    generated_text = tokenizer.decode(output_ids[0, inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    return generated_text.strip()

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Model and Tokenizer loaded successfully.


In [ ]:
train_file_path = '/content/trac2_CONVT_train.csv'
dev_file_path = '/content/trac2_CONVT_dev.csv'
test_file_path = '/content/goldstandard_CONVT_test.csv'
dev_subset_size = 40

def load_and_preprocess_data(file_path: str) -> List[Dict]:

    if not os.path.exists(file_path):
        print(f" Error: File not found at {file_path}.")
        return []

    print(f"Loading data from {file_path}...")

    try:
        df = pd.read_csv(
            file_path,
            engine='python',
            on_bad_lines='skip'
        )
    except Exception as e:
        print(f" Fatal Error during CSV reading: {e}")
        return []

    df.columns = df.columns.str.replace('"', '').str.strip()

    text_col = 'text'
    conv_id_col = 'conversation_id'
    turn_id_col = 'turn_id'

    speaker_col = None
    if 'speaker' in df.columns:
        speaker_col = 'speaker'
    elif 'speaker_id' in df.columns:
        speaker_col = 'speaker_id'

    if speaker_col is None or text_col not in df.columns or conv_id_col not in df.columns:
        print(f" Error: Critical columns missing. Speaker column not found. Available columns: {df.columns.tolist()}")
        return []

    # 3. Sort by conversation and turn number
    df = df.sort_values(by=[conv_id_col, turn_id_col])

    # 4. Group by conversation_id to get full conversations
    grouped = df.groupby(conv_id_col)
    processed_data = []

    # Data Restructuring Logic
    for conv_id, group in grouped:
        turns = []
        for index, row in group.iterrows():
            turns.append({
                'turn_id': row[turn_id_col],
                'speaker': str(row[speaker_col]).strip(),
                'text': str(row[text_col]).strip()
            })

        if len(turns) < 5:
            continue

        # 1. Input Context (Turns 1-5)
        context_turns = turns[:5]
        context_history = [(t['speaker'], t['text']) for t in context_turns]
        context_turns_5 = " ".join([Context_format.format(speaker=s, utterance=t) for s, t in context_history])

        # 2. Gold Output (Turns 6-15) - Max 10 turns
        gold_turns = turns[5:15]
        gold_history = [(t['speaker'], ' '.join(t['text'].split())) for t in gold_turns] # Simple clean-up
        turns_6_to_15 = " ".join([Context_format.format(speaker=s, utterance=t) for s, t in gold_history])

        processed_data.append({
            'id': conv_id,
            'context_turns_5': context_turns_5,
            'turns_6_to_15': turns_6_to_15,
            'context_history': context_history,
            'gold_history': gold_history
        })

    return processed_data

train_data = load_and_preprocess_data(train_file_path)
dev_data = load_and_preprocess_data(dev_file_path)
test_data = load_and_preprocess_data(test_file_path)

if train_data and dev_data and test_data:
    print(f"\n Data loaded successfully. Train={len(train_data)}, Dev={len(dev_data)}, Test={len(test_data)}")
    dev_subset = dev_data[:dev_subset_size]
    print(f"Using {len(dev_subset)} development examples for configuration testing.")
else:
    dev_subset = []
    print("\n Warning: Configuration testing skipped due to empty or failed data loading.")

Loading data from /content/trac2_CONVT_train.csv...
Loading data from /content/trac2_CONVT_dev.csv...
Loading data from /content/goldstandard_CONVT_test.csv...

 Data loaded successfully. Train=487, Dev=33, Test=67
Using 33 development examples for configuration testing.


In [ ]:
# def create_icl_prompt(unlabeled_instance: Dict, few_shot_examples: List[Dict], context_format: str = Context_format) -> str:

#     icl_prompt_parts = []

#     # 1. Add Few-Shot Examples (Input + Gold Output)
#     for i, example in enumerate(few_shot_examples):
#         input_context = example['context_turns_5']
#         gold_output = example['turns_6_to_15']
#         icl_prompt_parts.append(f"Example {i+1} \n{input_context} {gold_output}")

#     unlabeled_context = unlabeled_instance['context_turns_5']

#     final_prompt = "\n\n".join(icl_prompt_parts)
#     final_prompt += f"\n\n Conversation to Complete \n{unlabeled_context} "

#     return final_prompt

In [ ]:
def create_icl_prompt(unlabeled_instance: Dict, few_shot_examples: List[Dict], context_format: str = Context_format) -> str:
    icl_prompt_parts = []

    # Add instruction
    icl_prompt_parts.append("Task: Given the first 5 turns of a conversation, generate the next 10 turns (turns 6-15).\n")

    # Add Few-Shot Examples with clear INPUT/OUTPUT labels
    for i, example in enumerate(few_shot_examples):
        icl_prompt_parts.append(f"\n### Example {i+1} ###")
        icl_prompt_parts.append(f"INPUT (Turns 1-5):\n{example['context_turns_5']}")
        icl_prompt_parts.append(f"\nOUTPUT (Turns 6-15):\n{example['turns_6_to_15']}")

    # Add the unlabeled instance
    icl_prompt_parts.append(f"\n### Your Turn ###")
    icl_prompt_parts.append(f"INPUT (Turns 1-5):\n{unlabeled_instance['context_turns_5']}")
    icl_prompt_parts.append(f"\nOUTPUT (Turns 6-15):")

    return "\n".join(icl_prompt_parts)

In [ ]:
!pip install rouge_score
!pip install bert_score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=329a443521beb22cc0d9165bc031bc87a947dce9a02916642c381c144c56d3e9
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.6 MB/s eta 0:00:00


In [ ]:
def evaluate_config(num_shots: int, dev_data_subset: List[Dict]) -> Dict:
    print(f"\n Starting {num_shots}-shot evaluation on {len(dev_data_subset)} samples")

    if num_shots > len(train_data):
        return {'Shots': num_shots, 'ROUGE-L': 0, 'BLEU': 0, 'BertScore_F1': 0}

    few_shot_examples = random.sample(train_data, num_shots)

    generated_responses = []
    gold_responses = []
    failed_generations = 0
    PLACEHOLDER = "GENERATION_FAILED"

    for i, instance in enumerate(dev_data_subset):
        if i % 10 == 0:
             print(f"Processing sample {i+1}/{len(dev_data_subset)}...")

        prompt = create_icl_prompt(instance, few_shot_examples)

        generated = ""
        try:
            generated = generate_response(prompt)
        except Exception:
            generated = ""

        if not generated.strip():
            generated = PLACEHOLDER
            failed_generations += 1

        generated_responses.append(generated)
        gold_responses.append(instance['turns_6_to_15'])

    if failed_generations > 0:
        print(f" Warning: {failed_generations} predictions failed and were replaced with a placeholder.")

    bertscore_predictions = []
    bertscore_references = []

    for pred, ref in zip(generated_responses, gold_responses):
        if pred != PLACEHOLDER:
            bertscore_predictions.append(pred)
            bertscore_references.append(ref)

    rouge_results = rouge.compute(predictions=generated_responses, references=gold_responses)


    bleu_references = [[ref] for ref in gold_responses] # List of lists of strings
    bleu_results = bleu.compute(predictions=generated_responses, references=bleu_references)

    if bertscore_predictions:
        bert_results = bertscore.compute(
            predictions=bertscore_predictions,
            references=bertscore_references,
            model_type="distilbert-base-uncased",
            device=device
        )
        bert_f1 = sum(bert_results['f1']) / len(bert_results['f1'])
    else:
        bert_f1 = 0.0

    metrics = {
        'Shots': num_shots,
        'ROUGE-1': rouge_results['rouge1'],
        'ROUGE-2': rouge_results['rouge2'],
        'ROUGE-L': rouge_results['rougeL'],
        'BLEU': bleu_results['bleu'],
        'BertScore_F1': bert_f1,
    }

    return metrics

In [ ]:
!pip install tabulate
!pip install rouge_score
!pip install bert_score

In [ ]:
from evaluate import load
import random
import pandas as pd
from tabulate import tabulate

rouge = load("rouge")
bleu = load("bleu")
bertscore = load("bertscore")

results_3_shot = evaluate_config(num_shots=3, dev_data_subset=dev_subset)
results_5_shot = evaluate_config(num_shots=5, dev_data_subset=dev_subset)


all_results = [results_3_shot, results_5_shot]

headers = "keys"

table_output = tabulate(all_results, headers=headers, tablefmt="fancy_grid", floatfmt=".4f")

print("\n Consolidated Evaluation Results")
print(table_output)


 Starting 3-shot evaluation on 33 samples
Processing sample 1/33...


Both `max_new_tokens` (=500) and `max_length`(=1504) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=500) and `max_length`(=1464) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=500) and `max_length`(=1545) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=500) and `max_length`(=1513) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generati

Processing sample 11/33...


Both `max_new_tokens` (=500) and `max_length`(=1525) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=500) and `max_length`(=1584) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=500) and `max_length`(=1515) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=500) and `max_length`(=1548) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generati

Processing sample 21/33...


...

Processing sample 21/33...


Both `max_new_tokens` (=500) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=500) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=500) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=500) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generati

Processing sample 31/33...


Both `max_new_tokens` (=500) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=500) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



 Consolidated Evaluation Results
╒═════════╤═══════════╤═══════════╤═══════════╤════════╤════════════════╕
│   Shots │   ROUGE-1 │   ROUGE-2 │   ROUGE-L │   BLEU │   BertScore_F1 │
╞═════════╪═══════════╪═══════════╪═══════════╪════════╪════════════════╡
│       3 │    0.2856 │    0.0541 │    0.1364 │ 0.0380 │         0.7570 │
├─────────┼───────────┼───────────┼───────────┼────────┼────────────────┤
│       5 │    0.2468 │    0.0360 │    0.1124 │ 0.0153 │         0.7118 │
╘═════════╧═══════════╧═══════════╧═══════════╧════════╧════════════════╛


In [ ]:
def parse_generated_turns(generated_text: str, start_turn_id: int = 6) -> List[Dict]:

    pattern = r"([A-Za-z0-9_]+)\s*:\s*(.*?)(?=([A-Za-z0-9_]+)\s*:|$)"

    turns = []
    current_turn = start_turn_id

    matches = re.findall(pattern, generated_text, re.DOTALL)

    for speaker, utterance, next_speaker in matches:
        if utterance.strip():
            turns.append({
                'turn_id': current_turn,
                'speaker': speaker.strip(),
                'text': utterance.strip()
            })
            current_turn += 1

    return turns[:10]

def run_test_inference_and_save(test_data: List[Dict], few_shot_examples: List[Dict], output_filename: str = 'submission_predictions.csv'):

    print(f"\n Running final inference on {len(test_data)} test samples")

    final_predictions = []

    for i, instance in enumerate(test_data):
        if i % 50 == 0:
            print(f"Processing test sample {i+1}/{len(test_data)}...")

        # 1. Generate ICL Prompt
        prompt = create_icl_prompt(instance, few_shot_examples)

        # 2. Generate Response
        generated_text = ""
        try:
            generated_text = generate_response(prompt)
        except Exception as e:
            print(f"Generation failed for ID {instance['id']}: {e}")
            generated_text = ""


        parsed_turns = parse_generated_turns(generated_text)

        for turn in parsed_turns:
            formatted_response = f"{turn['speaker']}: {turn['text']}"

            final_predictions.append({
                'id': instance['id'],
                'turn number': turn['turn_id'],
                'generated response': formatted_response
            })

    if final_predictions:
        df_results = pd.DataFrame(final_predictions)

        df_results = df_results[['id', 'turn number', 'generated response']]
        df_results.to_csv(output_filename, index=False)
        print(f"\n Successfully generated predictions and saved to: {output_filename}")
    else:
        print("\n Warning: No predictions were generated. CSV file not created.")



In [ ]:
import random
import pandas as pd

if 'train_data' in locals() and 'test_data' in locals() and train_data and test_data:

    best_shot_config = 5

    try:
        final_few_shot_examples = random.sample(train_data, best_shot_config)
    except ValueError:
        print(f" Cannot sample {best_shot_config} shots. Using all available training data.")
        final_few_shot_examples = train_data

    print(f"\n Starting final {best_shot_config}-shot inference on the Test Dataset")

    # 2. Call the main function to run inference and save the CSV
    run_test_inference_and_save(
        test_data=test_data,
        few_shot_examples=final_few_shot_examples,
        output_filename='final_submission_predictions.csv'
    )
else:
    print("\n Error: Cannot run test inference. Ensure 'train_data' and 'test_data' are loaded correctly from your CSV files.")

Both `max_new_tokens` (=500) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



 Starting final 5-shot inference on the Test Dataset

 Running final inference on 67 test samples
Processing test sample 1/67...


Both `max_new_tokens` (=500) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=500) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=500) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=500) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generati

Processing test sample 51/67...


Both `max_new_tokens` (=500) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=500) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=500) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=500) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generati


 Successfully generated predictions and saved to: final_submission_predictions.csv
